In [49]:
from remove_duplicates import *
%reload_ext autoreload
%autoreload 2

In [81]:
org_df = None
with pymysql.connect(**CONN_INFO) as cursor:
        query = """SELECT oid, name
                   FROM Organizations"""
        cursor.execute(query)

        org_lists = [[oid, name] for oid, name in cursor]
        org_df = pd.DataFrame(org_lists, columns=['oid', 'name'])


In [24]:
# org_df = org_df.sample(1000)

In [82]:
org_df['normed_name'] = org_df['name'].apply(norm_name)
org_df['hard_coded_name'] = org_df['normed_name'].apply(hard_coded_matches)

In [85]:
cpy_org_df = org_df.copy()

In [86]:
org_df['matched_indices'] = org_df.apply(lambda row: match_other_rows(row, cpy_org_df), axis=1)

In [87]:
# Yes, this needs to happen twice
cpy_org_df = org_df.copy()

In [88]:
org_df['delete'] = org_df.apply(lambda row: mark_rows_for_deletion(row, cpy_org_df), axis=1)

In [89]:
cpy_org_df = org_df.copy()

In [102]:
cpy_org_df[cpy_org_df.oid == 20172]

,oid,name,normed_name,hard_coded_name,matched_indices,delete
12495,20172,Milberg Weiss Bershad Hynes & Lerach,milberg weiss bershad hynes & lerach,None,{},True


In [90]:
org_df = org_df[~org_df['delete']]

In [91]:
# The following code is used for making your duplicate spreadsheet
org_df = org_df[org_df['matched_indices'].apply(len) > 0]

In [100]:
org_df[org_df['matched_indices'].apply(len) == org_df['matched_indices'].apply(len).max()]

,oid,name,normed_name,hard_coded_name,matched_indices,delete
10495,13561,At&T Inc. And Its Affiliates,at&t inc. and its affiliates,None,"{38402, 34822, 40970, 40971, 30220, 68624, 507...",False


In [92]:
output = org_df.apply(lambda row: join_dup_org_names(row, cpy_org_df), axis=1)

In [93]:
mx = org_df['matched_indices'].apply(len).max()

In [94]:
columns = ['name',
           'hard_coded_name',
           'normed_name',
           'oid']
for i in range(1, mx+1):
    columns.append('name_' + str(i))
    columns.append('oid_' + str(i))

In [95]:
output = output[columns]

In [96]:
output.to_csv('duplicate_removal_final.csv', index=False)

In [13]:
%%timeit
d = org_df['name'].apply(lambda name: l_distance(name, 'Sierra Club'))

10 loops, best of 3: 126 ms per loop
